In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
import tensorflow as tf
from video.data import read_dataset

In [3]:
tf.enable_eager_execution()

# カテゴリ一覧

In [4]:
!gsutil cp gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv .

Copying gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [5]:
import csv

label_map = {}
with open('./vocabulary.csv', 'r') as c:
    reader = csv.reader(c)
    next(reader, None)
    for r in reader:
        label_map[r[0]] = r[3]

# 評価データセットの読み込み

In [6]:
data = read_dataset(
    'gs://asl-mixi-project-bucket/data/youtube-8m/valid/*.tfrecord',
    tf.estimator.ModeKeys.EVAL,
    batch_size=5
).shuffle(1024)

# 推論

In [7]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

PROJECT = 'qwiklabs-gcp-ml-83914069970b'
MODEL_NAME = 'youtube8mkojo'
VERSION = 'v1'

credentials = GoogleCredentials.get_application_default()
api = discovery.build(
    "ml",
    "v1",
    credentials = credentials,
    discoveryServiceUrl = "https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json"
)

def convert_data(data):
    batch, labels = next(iter(data))
    batch.pop('id')
    feature = [{'mean_rgb': r.numpy().tolist(), 'mean_audio': a.numpy().tolist()} for r, a in zip(batch['mean_rgb'], batch['mean_audio'])]
    labels = [[label_map[str(i)] for i, v in enumerate(label) if v] for label in labels]
    return feature, labels

def show_predict(prediction, label):
    print('labels:', label)
    for i, index in enumerate(prediction["predicted_topk"]):
        print("{}th({}:{}) probality is {}".format(
            (i+1),
            index,
            label_map["{}".format(index)],
            prediction['probabilities'][index]
        ))
    print('')

def predict(model_name=MODEL_NAME, version=VERSION):
    feature, labels = convert_data(data)
    request_data = {"instances": feature}
    parent = "projects/{}/models/{}/versions/{}".format(PROJECT,MODEL_NAME, VERSION)
    response = api.projects().predict(body = request_data, name = parent).execute()
    for p, l in zip(response['predictions'], labels):
        show_predict(p, l)
    return None

In [8]:
predict('youtube8mkojo', 'v1')

labels: ['Vehicle', 'Car', 'Sports car', 'Supercar', 'Slide show', 'Audi']
1th(2:Vehicle) probality is 0.9885274171829224
2th(7:Car) probality is 0.9848949909210205
3th(48:Sports car) probality is 0.880866527557373
4th(213:Coupé) probality is 0.5943225622177124
5th(177:Supercar) probality is 0.35532230138778687

labels: ['Airplane', 'Extreme sport', 'Parachuting', 'Parachute']
1th(398:Parachuting) probality is 0.9763474464416504
2th(501:Parachute) probality is 0.9586477279663086
3th(166:Extreme sport) probality is 0.5617824196815491
4th(333:Jumping) probality is 0.25855499505996704
5th(44:Aircraft) probality is 0.015628278255462646

labels: ['Dental braces']
1th(11:Food) probality is 0.7975030541419983
2th(32:Dish (food)) probality is 0.2644324004650116
3th(29:Cuisine) probality is 0.2557116150856018
4th(368:Eating) probality is 0.1173916757106781
5th(20:Recipe) probality is 0.10123005509376526

labels: ['Concert', 'Performance art', 'Dance', 'Resort', 'Suite (hotel)', 'Spa']
1th(3:Con